In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6330112282498742237
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7308640373012729186
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1406762599
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8562945063743031072
physical_device_desc: "device: 0, name: GeForce GT 1030, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11805408242213347346
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
import tensorflow as tf
GPU_list = tf.config.list_physical_devices('GPU')
GPU_list

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization
from keras import backend as K
import keras

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 20025
nb_validation_samples = 4000
epochs = 20
batch_size = 16
opt = keras.optimizers.Adam(learning_rate=0.00001)

if K.image_data_format() == 'channels_first':
    input_shape = (1 ,img_width, img_height)
else:
    input_shape = (img_width, img_height, 1)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
path= "./PF_Data/"
train_datagen = ImageDataGenerator( rescale=1./255.0,
                                    rotation_range=20,
                                    zoom_range=0.15,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.15,
                                    horizontal_flip=True,
                                    fill_mode="nearest")
valid_datagen = ImageDataGenerator(rescale=1./255.0)
test_datagen = ImageDataGenerator(rescale=1./255.0)

In [11]:
train_generator = train_datagen.flow_from_directory(
    directory=path+"Train/",
    target_size=(img_width, img_height),
    color_mode="grayscale",
    batch_size=16,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 20025 images belonging to 2 classes.


In [12]:
valid_generator = valid_datagen.flow_from_directory(
    directory=path+"Valid/",
    target_size=(img_width, img_height),
    color_mode="grayscale",
    batch_size=16,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 4000 images belonging to 2 classes.


In [13]:
train_generator.reset()
valid_generator.reset()

In [14]:
       model.fit( 
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=5,
            validation_data=valid_generator,
            validation_steps=nb_validation_samples // batch_size,
            verbose = 2)

Epoch 1/5
1251/1251 - 557s - loss: 0.1396 - accuracy: 0.9510 - val_loss: 0.0311 - val_accuracy: 0.9915
Epoch 2/5
1251/1251 - 549s - loss: 0.0769 - accuracy: 0.9735 - val_loss: 0.0422 - val_accuracy: 0.9870
Epoch 3/5
1251/1251 - 550s - loss: 0.0562 - accuracy: 0.9818 - val_loss: 0.0237 - val_accuracy: 0.9930
Epoch 4/5
1251/1251 - 550s - loss: 0.0507 - accuracy: 0.9839 - val_loss: 0.0172 - val_accuracy: 0.9950
Epoch 5/5
1251/1251 - 554s - loss: 0.0432 - accuracy: 0.9855 - val_loss: 0.0234 - val_accuracy: 0.9937


In [15]:
model.evaluate(valid_generator,
steps=nb_validation_samples // batch_size)

250/250 [==============================] - 75s 299ms/step - loss: 0.0234 - accuracy: 0.9937


[0.02341511845588684, 0.9937499761581421]

In [29]:
model.save_weights('my_model_weights.h5')

In [21]:
test_generator = test_datagen.flow_from_directory(
    directory=path+"test/",
    target_size=(150, 150),
    color_mode="grayscale",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 4 images belonging to 1 classes.


In [22]:
STEP_SIZE_TEST=4
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

4/4 [==============================] - 0s 30ms/step


In [23]:
pred

array([[1.4534596e-04],
       [3.8973594e-01],
       [9.9997771e-01],
       [9.9993205e-01]], dtype=float32)

In [34]:
import numpy as np
np.round(pred,3)
for i in pred:
    if(i < 0.5):
        print("HB whale: " + str(pred[i]))
    else:
        print("Right whale: " + str(pred[i]))

HB whale
HB whale
Right whale
Right whale
